In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,confusion_matrix, cohen_kappa_score, precision_score, recall_score
from sklearn.metrics import precision_recall_curve, auc, average_precision_score, f1_score, roc_auc_score, roc_curve
import pandas as pd
import math
from joblib import dump, load
import random
import pickle
import scipy.stats
from PIL import Image
import scipy.stats as stats

from Functions import post_processing
from Functions import general_functions
from Functions import feature_creation

from skimage.restoration import denoise_bilateral

import operator

In [2]:
zone_1_path = "dataset/zone_4.pickle"
zone_2_path = "dataset/zone_6.pickle"
zone_3_path = "dataset/zone_9.pickle"

In [3]:
file = open(zone_1_path, "rb")
zone_1 = pickle.load(file)

file = open(zone_2_path, "rb")
zone_2 = pickle.load(file)

file = open(zone_3_path, "rb")
zone_3 = pickle.load(file)

file = None

In [4]:
#zone_1_resampled = general_functions.create_balanced_dataset([zone_1_path])
#zone_2_resampled = general_functions.create_balanced_dataset([zone_2_path])
#zone_3_resampled = general_functions.create_balanced_dataset([zone_3_path])

#with open("dataset/zone_1_resampled.pickle", "wb") as file:
#    pickle.dump(zone_1_resampled, file)
#with open("dataset/zone_2_resampled.pickle", "wb") as file:
#    pickle.dump(zone_2_resampled, file)
#with open("dataset/zone_3_resampled.pickle", "wb") as file:
#    pickle.dump(zone_3_resampled, file)

In [5]:
with open("dataset/zone_1_resampled.pickle", "rb") as file:
    zone_1_resampled = pickle.load(file)
with open("dataset/zone_2_resampled.pickle", "rb") as file:
    zone_2_resampled = pickle.load(file)
with open("dataset/zone_3_resampled.pickle", "rb") as file:
    zone_3_resampled = pickle.load(file)

In [6]:
zone_1.shape

(7852140, 82)

In [7]:
experiment_arr = [(pd.concat([zone_1_resampled, zone_3_resampled], ignore_index=True), zone_2),
                  (pd.concat([zone_1_resampled, zone_2_resampled], ignore_index=True), zone_3)]

In [8]:
zone_1 = None
zone_2 = None
zone_3 = None
zone_1_resampled = None
zone_2_resampled = None
zone_3_resampled = None

In [19]:
most_important_features = experiment_arr[0][0].columns.tolist()[1:]
for num_features in [81, 70, 60, 50, 40]:
    
    most_important_features = most_important_features[:num_features]
    print(f"Features used in experiment:\n{most_important_features}")
    print("\n")
    feature_importances = {i:0 for i in most_important_features}
    y_test_all = []
    pred_all = []
    
    
    for (training_dataset, test_dataset) in experiment_arr:

        X_train = training_dataset.filter(items=most_important_features).loc[:, training_dataset.filter(items=most_important_features).columns != "label_3m"]
        y_train = training_dataset["label_3m"]
        training_dataset = None

        _max_features = 25 if num_features > 25 else num_features
        clf = RandomForestClassifier(n_estimators=200, class_weight="balanced", max_features=_max_features,
                                     random_state=42, n_jobs=-1)

        clf.fit(X_train, y_train)

        X_test = test_dataset.filter(items=most_important_features).loc[:, test_dataset.filter(items=most_important_features).columns != "label_3m"]
        y_test = test_dataset["label_3m"]
        
        print(f"Amount of features X_train: {len(X_train.columns)}, X_test: {len(X_test.columns)}")

        proba = clf.predict_proba(X_test)[:,1:].reshape(2997,2620)
        
        proba_post_process = post_processing.proba_post_process(proba, 6, 0.4)
    
        labels_grid = post_processing.raster_to_zones(np.array(y_test).reshape(2997, 2620), 6, 4)

        for i, pred_var in enumerate(proba_post_process.reshape(-1)):
            y_test_all.append(labels_grid.reshape(-1)[i])
            pred_all.append(pred_var)

        importances = clf.feature_importances_

        
        for i, importance in enumerate(importances):
            feature_importances[most_important_features[i]] += importance
        
    for importance_name in most_important_features:
        feature_importances[importance_name] /= 2
    
    
    
    most_important_features = list(dict(sorted(feature_importances.items(),
                                               key=operator.itemgetter(1),reverse=True)).keys())

    print("\n")
    print(f"Importances for experiment:")
    for key in most_important_features:
        print(key, " - ", feature_importances[key])
    print("\n")
    print(f"Amount of features used:   {num_features}")
    print("Cohen's kappa score        ", cohen_kappa_score(y_test_all, pred_all))
    print("Accuracy score             ", accuracy_score(y_test_all, pred_all))
    print("Recall score               ", recall_score(y_test_all, pred_all))
    print("Precision score            ", precision_score(y_test_all, pred_all))
    print("\n------------------------------------------------------\n")

Features used in experiment:
['hpmf_raw', 'skyview_raw', 'impundment_raw', 'slope_raw', 'DEM_ditch_detection', 'DEM_ditch_detection_no_streams', 'conic_mean', 'skyview_non_ditch', 'skyview_gabor', 'conic_mean_no_streams', 'skyview_gabor_no_streams', 'skyview_mean_2', 'skyview_mean_3', 'skyview_mean_4', 'skyview_mean_6', 'skyview_median_2', 'skyview_median_4', 'skyview_median_6', 'skyview_min_2', 'skyview_min_4', 'skyview_min_6', 'skyview_max_2', 'skyview_max_4', 'skyview_max_6', 'skyview_std_2', 'skyview_std_4', 'skyview_std_6', 'impundment_amplified', 'impoundment_amplified_no_streams', 'impundment_mean_2', 'impundment_mean_3', 'impundment_mean_4', 'impundment_mean_6', 'impundment_median_2', 'impundment_median_4', 'impundment_median_6', 'impundment_min_2', 'impundment_min_4', 'impundment_min_6', 'impundment_max_2', 'impundment_max_4', 'impundment_max_6', 'impundment_std_2', 'impundment_std_4', 'impundment_std_6', 'hpmf_filter', 'hpmf_gabor', 'hpmf_gabor_no_streams', 'hpmf_filter_no_st

Cohen's kappa score         0.6074974677335433
Accuracy score              0.9842839022228336
Recall score                0.5781367986653789
Precision score             0.6579925650557621

------------------------------------------------------

Features used in experiment:
['impundment_mean_3', 'impundment_mean_4', 'impundment_mean_2', 'hpmf_mean_4', 'hpmf_mean_3', 'impundment_median_4', 'hpmf_min_2', 'impundment_amplified', 'impoundment_amplified_no_streams', 'hpmf_median_4', 'skyview_max_6', 'skyview_gabor', 'impundment_std_4', 'skyview_gabor_no_streams', 'slope_std_6', 'impundment_median_2', 'skyview_max_4', 'slope_min_6', 'hpmf_mean_6', 'impundment_mean_6', 'skyview_non_ditch', 'slope_min_4', 'impundment_max_6', 'impundment_max_2', 'hpmf_min_4', 'hpmf_mean_2', 'slope_median_6', 'slope_non_ditch', 'impundment_std_6', 'hpmf_filter', 'impundment_median_6', 'skyview_max_2', 'hpmf_filter_no_streams', 'skyview_min_6', 'impundment_max_4', 'hpmf_gabor_no_streams', 'slope_mean_6', 'slope_st

Amount of features X_train: 40, X_test: 40
Amount of features X_train: 40, X_test: 40


Importances for experiment:
impundment_mean_3  -  0.22955582521855084
impundment_mean_4  -  0.12024382036414162
hpmf_mean_4  -  0.030181728607523647
impundment_amplified  -  0.028162619232376652
hpmf_median_4  -  0.027641721435081604
impundment_mean_2  -  0.026668935044070093
impoundment_amplified_no_streams  -  0.02426657240282676
skyview_max_6  -  0.02189465348457963
slope_std_6  -  0.020489618137113676
hpmf_mean_3  -  0.01806767137202291
skyview_max_4  -  0.01743659389469335
slope_min_6  -  0.017415744121230314
hpmf_mean_6  -  0.017250574352831345
skyview_gabor_no_streams  -  0.017122958016857495
hpmf_min_4  -  0.016838186083621715
skyview_gabor  -  0.01675801012920184
impundment_max_6  -  0.016529549983806343
skyview_median_6  -  0.01645913950140155
skyview_non_ditch  -  0.01641407015374431
slope_median_6  -  0.016371222499315323
slope_min_4  -  0.01566820129883951
impundment_median_2  -  0.0154

In [20]:
most_important_features = experiment_arr[0][0].columns.tolist()[1:]
for num_features in [81, 78, 76, 74, 72, 70, 68, 66, 64, 62, 60]:
    
    most_important_features = most_important_features[:num_features]
    print(f"Features used in experiment:\n{most_important_features}")
    print("\n")
    feature_importances = {i:0 for i in most_important_features}
    y_test_all = []
    pred_all = []
    
    
    for (training_dataset, test_dataset) in experiment_arr:

        X_train = training_dataset.filter(items=most_important_features).loc[:, training_dataset.filter(items=most_important_features).columns != "label_3m"]
        y_train = training_dataset["label_3m"]
        training_dataset = None

        _max_features = 25 if num_features > 25 else num_features
        clf = RandomForestClassifier(n_estimators=200, class_weight="balanced", max_features=_max_features,
                                     random_state=42, n_jobs=-1)

        clf.fit(X_train, y_train)

        X_test = test_dataset.filter(items=most_important_features).loc[:, test_dataset.filter(items=most_important_features).columns != "label_3m"]
        y_test = test_dataset["label_3m"]
        
        print(f"Amount of features X_train: {len(X_train.columns)}, X_test: {len(X_test.columns)}")

        proba = clf.predict_proba(X_test)[:,1:].reshape(2997,2620)
        
        proba_post_process = post_processing.proba_post_process(proba, 6, 0.4)
    
        labels_grid = post_processing.raster_to_zones(np.array(y_test).reshape(2997, 2620), 6, 4)

        for i, pred_var in enumerate(proba_post_process.reshape(-1)):
            y_test_all.append(labels_grid.reshape(-1)[i])
            pred_all.append(pred_var)

        importances = clf.feature_importances_

        
        for i, importance in enumerate(importances):
            feature_importances[most_important_features[i]] += importance
        
    for importance_name in most_important_features:
        feature_importances[importance_name] /= 2
    
    
    
    most_important_features = list(dict(sorted(feature_importances.items(),
                                               key=operator.itemgetter(1),reverse=True)).keys())

    print("\n")
    print(f"Importances for experiment:")
    for key in most_important_features:
        print(key, " - ", feature_importances[key])
    print("\n")
    print(f"Amount of features used:   {num_features}")
    print("Cohen's kappa score        ", cohen_kappa_score(y_test_all, pred_all))
    print("Accuracy score             ", accuracy_score(y_test_all, pred_all))
    print("Recall score               ", recall_score(y_test_all, pred_all))
    print("Precision score            ", precision_score(y_test_all, pred_all))
    print("\n------------------------------------------------------\n")

Features used in experiment:
['hpmf_raw', 'skyview_raw', 'impundment_raw', 'slope_raw', 'DEM_ditch_detection', 'DEM_ditch_detection_no_streams', 'conic_mean', 'skyview_non_ditch', 'skyview_gabor', 'conic_mean_no_streams', 'skyview_gabor_no_streams', 'skyview_mean_2', 'skyview_mean_3', 'skyview_mean_4', 'skyview_mean_6', 'skyview_median_2', 'skyview_median_4', 'skyview_median_6', 'skyview_min_2', 'skyview_min_4', 'skyview_min_6', 'skyview_max_2', 'skyview_max_4', 'skyview_max_6', 'skyview_std_2', 'skyview_std_4', 'skyview_std_6', 'impundment_amplified', 'impoundment_amplified_no_streams', 'impundment_mean_2', 'impundment_mean_3', 'impundment_mean_4', 'impundment_mean_6', 'impundment_median_2', 'impundment_median_4', 'impundment_median_6', 'impundment_min_2', 'impundment_min_4', 'impundment_min_6', 'impundment_max_2', 'impundment_max_4', 'impundment_max_6', 'impundment_std_2', 'impundment_std_4', 'impundment_std_6', 'hpmf_filter', 'hpmf_gabor', 'hpmf_gabor_no_streams', 'hpmf_filter_no_st

Cohen's kappa score         0.6035136831363332
Accuracy score              0.9841417753631494
Recall score                0.5738168408113091
Precision score             0.6546459911047001

------------------------------------------------------

Features used in experiment:
['impundment_mean_3', 'impundment_mean_4', 'impundment_median_4', 'impundment_mean_2', 'hpmf_mean_4', 'hpmf_mean_3', 'hpmf_min_2', 'impundment_amplified', 'impoundment_amplified_no_streams', 'hpmf_median_4', 'hpmf_mean_2', 'skyview_max_6', 'impundment_std_4', 'skyview_gabor', 'impundment_median_2', 'skyview_gabor_no_streams', 'impundment_max_2', 'slope_std_6', 'slope_min_6', 'skyview_max_4', 'hpmf_mean_6', 'impundment_mean_6', 'skyview_non_ditch', 'slope_min_4', 'impundment_max_6', 'slope_median_6', 'impundment_std_6', 'slope_non_ditch', 'hpmf_min_4', 'hpmf_filter', 'hpmf_filter_no_streams', 'impundment_median_6', 'impundment_max_4', 'skyview_min_6', 'skyview_max_2', 'hpmf_median_2', 'slope_std_4', 'slope_mean_6', 'h

Cohen's kappa score         0.6037633512992364
Accuracy score              0.9841830379998319
Recall score                0.572868557379928
Precision score             0.6564046842931305

------------------------------------------------------

Features used in experiment:
['impundment_mean_3', 'impundment_mean_4', 'impundment_mean_2', 'hpmf_mean_3', 'hpmf_mean_4', 'impundment_median_4', 'hpmf_min_2', 'impundment_amplified', 'impoundment_amplified_no_streams', 'impundment_max_2', 'hpmf_median_4', 'skyview_max_6', 'skyview_gabor', 'impundment_std_4', 'skyview_gabor_no_streams', 'impundment_median_2', 'slope_std_6', 'skyview_max_4', 'hpmf_mean_6', 'slope_min_6', 'impundment_mean_6', 'skyview_non_ditch', 'slope_min_4', 'hpmf_min_4', 'impundment_max_6', 'hpmf_mean_2', 'slope_median_6', 'impundment_std_2', 'impundment_std_6', 'slope_non_ditch', 'hpmf_filter', 'impundment_median_6', 'hpmf_filter_no_streams', 'skyview_min_6', 'skyview_max_2', 'impundment_max_4', 'slope_mean_6', 'slope_std_4', 

Cohen's kappa score         0.5959455126570885
Accuracy score              0.9838621063811903
Recall score                0.5660198437088418
Precision score             0.6477752321234777

------------------------------------------------------

Features used in experiment:
['impundment_mean_3', 'impundment_mean_4', 'impundment_mean_2', 'hpmf_mean_4', 'hpmf_mean_3', 'impundment_median_4', 'hpmf_min_2', 'impundment_amplified', 'impoundment_amplified_no_streams', 'hpmf_median_4', 'skyview_max_6', 'skyview_gabor', 'impundment_max_2', 'skyview_gabor_no_streams', 'impundment_std_4', 'slope_std_6', 'skyview_max_4', 'hpmf_mean_6', 'slope_min_6', 'impundment_median_2', 'skyview_non_ditch', 'impundment_max_6', 'impundment_mean_6', 'slope_min_4', 'hpmf_min_4', 'slope_median_6', 'impundment_std_6', 'hpmf_mean_2', 'slope_non_ditch', 'hpmf_filter', 'impundment_median_6', 'skyview_max_2', 'hpmf_filter_no_streams', 'skyview_min_6', 'impundment_max_4', 'skyview_median_6', 'slope_mean_6', 'slope_std_4',

Cohen's kappa score         0.6063989150532858
Accuracy score              0.9842770251167198
Recall score                0.5757134076740714
Precision score             0.6586306653809064

------------------------------------------------------

Features used in experiment:
['impundment_mean_3', 'impundment_mean_4', 'hpmf_mean_4', 'impundment_mean_2', 'hpmf_mean_3', 'impundment_amplified', 'impundment_median_4', 'impoundment_amplified_no_streams', 'hpmf_median_4', 'hpmf_min_2', 'skyview_max_6', 'impundment_median_2', 'skyview_gabor', 'skyview_gabor_no_streams', 'slope_std_6', 'impundment_std_4', 'skyview_max_4', 'slope_min_6', 'impundment_max_2', 'hpmf_mean_6', 'skyview_non_ditch', 'impundment_mean_6', 'slope_min_4', 'hpmf_min_4', 'impundment_max_6', 'slope_median_6', 'slope_non_ditch', 'impundment_median_6', 'hpmf_filter', 'impundment_std_6', 'skyview_max_2', 'hpmf_filter_no_streams', 'skyview_min_6', 'impundment_max_4', 'skyview_median_6', 'slope_mean_6', 'slope_std_4', 'hpmf_mean_2',

Cohen's kappa score         0.6016658284447876
Accuracy score              0.9841211440448082
Recall score                0.5701290719114935
Precision score             0.6552434003390651

------------------------------------------------------

Features used in experiment:
['impundment_mean_3', 'impundment_mean_4', 'hpmf_mean_4', 'impundment_mean_2', 'hpmf_mean_3', 'impundment_median_4', 'impundment_amplified', 'impoundment_amplified_no_streams', 'hpmf_min_2', 'hpmf_median_4', 'skyview_max_6', 'skyview_gabor', 'skyview_gabor_no_streams', 'impundment_std_4', 'slope_std_6', 'skyview_max_4', 'slope_min_6', 'hpmf_mean_6', 'impundment_median_2', 'skyview_non_ditch', 'slope_min_4', 'impundment_max_6', 'impundment_mean_6', 'hpmf_min_4', 'slope_median_6', 'skyview_max_2', 'impundment_median_6', 'slope_non_ditch', 'hpmf_mean_2', 'hpmf_filter', 'impundment_std_6', 'hpmf_filter_no_streams', 'skyview_min_6', 'skyview_median_6', 'slope_std_4', 'slope_mean_6', 'hpmf_gabor', 'impundment_max_4', 'slop

In [7]:
from itertools import product

In [8]:
params = [["gini", "entropy"],
          [35, None],
          [2, 10],
          ["balanced", None],
          [200, 300]]

In [9]:
most_important_features = ['impundment_mean_3', 'impundment_mean_4', 'impundment_median_4', 'impundment_mean_2',
                           'impundment_amplified', 'hpmf_mean_4', 'impoundment_amplified_no_streams', 'hpmf_median_4',
                           'skyview_max_6', 'skyview_gabor', 'skyview_non_ditch', 'slope_min_6', 'skyview_max_4',
                           'skyview_gabor_no_streams', 'impundment_std_4', 'impundment_max_6', 'slope_non_ditch',
                           'hpmf_filter', 'hpmf_mean_3', 'hpmf_filter_no_streams', 'hpmf_mean_6', 'impundment_median_6',
                           'impundment_median_2', 'slope_std_6', 'impundment_mean_6', 'slope_median_6', 'slope_min_4',
                           'hpmf_min_2', 'impundment_std_6', 'skyview_median_6', 'skyview_min_6', 'slope_mean_6',
                           'impundment_max_2', 'skyview_std_6', 'slope_min_2', 'skyview_max_2', 'hpmf_min_4',
                           'impundment_max_4', 'slope_std_4', 'hpmf_gabor', 'hpmf_std_6', 'slope_max_4',
                           'hpmf_gabor_no_streams', 'skyview_min_4', 'slope_median_4', 'hpmf_mean_2', 'slope_median_2',
                           'slope_std_2', 'hpmf_min_6', 'hpmf_max_6', 'skyview_mean_6', 'skyview_median_4', 'hpmf_std_4']

In [10]:
param_combos = list(product(*params))

In [11]:
len(param_combos)

144

In [12]:
zone_4 = None
zone_7 = None
zone_4_resampled = None
zone_7_resampled = None
params_and_score = []
j = 0
for combo in param_combos:
    j += 1
    y_test_all = []
    pred_all = []
    
    params = {'criterion': combo[0],
    'max_depth': combo[1],
    'min_samples_split': combo[2],
    'class_weight': combo[3],
    'n_estimators': combo[4]}
    print(f"params: {params}")
    
    for (training_dataset, test_dataset) in experiment_arr:

        X_train = training_dataset.filter(items=most_important_features).loc[:, training_dataset.filter(items=most_important_features).columns != "label_3m"]
        y_train = training_dataset["label_3m"]
        training_dataset = None

        clf = RandomForestClassifier(**params, n_jobs=-1, random_state=89)
        print(clf)

        clf.fit(X_train, y_train)

        X_test = test_dataset.filter(items=most_important_features).loc[:, test_dataset.filter(items=most_important_features).columns != "label_3m"]
        y_test = test_dataset["label_3m"]
        
        print(f"Amount of features X_train: {len(X_train.columns)}, X_test: {len(X_test.columns)}")

        pred = clf.predict(X_test)

        for i, pred_var in enumerate(pred):
            y_test_all.append(y_test[i])
            pred_all.append(pred_var)

    print("\n")
    print(f"Iteration {j} of {len(param_combos)}")
    print("Cohen's kappa score        ", cohen_kappa_score(y_test_all, pred_all))
    print("Accuracy score             ", accuracy_score(y_test_all, pred_all))
    print("Recall score               ", recall_score(y_test_all, pred_all))
    print("Precision score            ", precision_score(y_test_all, pred_all))
    print("\n------------------------------------------------------\n")
    params_and_score.append((params, cohen_kappa_score(y_test_all, pred_all)))

print(params_and_score)

params: {'criterion': 'gini', 'max_depth': 35, 'max_features': 'log2', 'min_samples_split': 2, 'class_weight': 'balanced', 'n_estimators': 100}
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=35, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=89, verbose=0,
                       warm_start=False)
Amount of features X_train: 53, X_test: 53
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=35, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0,

Amount of features X_train: 53, X_test: 53
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                       class_weight='balanced_subsample', criterion='gini',
                       max_depth=35, max_features='log2', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=300, n_jobs=-1, oob_score=False,
                       random_state=89, verbose=0, warm_start=False)
Amount of features X_train: 53, X_test: 53


Iteration 6 of 144
Cohen's kappa score         0.41765441554749516
Accuracy score              0.9563667993693439
Recall score                0.7229061927535719
Precision score             0.31281337933195213

------------------------------------------------------

params: {'criterion': 'gini', 'max_depth': 35, 'max_features': 'log2', 'min_samples_split

Amount of features X_train: 53, X_test: 53


Iteration 11 of 144
Cohen's kappa score         0.42227894657701326
Accuracy score              0.9571729490304554
Recall score                0.7222964715105027
Precision score             0.3174507089937469

------------------------------------------------------

params: {'criterion': 'gini', 'max_depth': 35, 'max_features': 'log2', 'min_samples_split': 10, 'class_weight': 'balanced', 'n_estimators': 300}
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=35, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=-1, oob_score=False, random_state=89, verbose=0,
                       warm_start=Fal

Recall score                0.7201107297328931
Precision score             0.31336987962938834

------------------------------------------------------

params: {'criterion': 'gini', 'max_depth': 35, 'max_features': 'log2', 'min_samples_split': 10, 'class_weight': None, 'n_estimators': 200}
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=35, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=-1, oob_score=False, random_state=89, verbose=0,
                       warm_start=False)
Amount of features X_train: 53, X_test: 53
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                       class_weight='balanced_subsample', criterion='gini',
                       max_depth=35, max_features='sqrt', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=-1, oob_score=False,
                       random_state=89, verbose=0, warm_start=False)
Amount of features X_train: 53, X_test: 53
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                       class_weight='balanced_subsample', criterion='gini',
                       max_depth=35, max_features='sqrt', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_

...

params: {'criterion': 'entropy', 'max_depth': None, 'max_features': 'log2', 'min_samples_split': 10, 'class_weight': 'balanced_subsample', 'n_estimators': 200}
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                       class_weight='balanced_subsample', criterion='entropy',
                       max_depth=None, max_features='log2', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=10, min_weight_fraction_leaf=0.0,
                       n_estimators=200, n_jobs=-1, oob_score=False,
                       random_state=89, verbose=0, warm_start=False)
Amount of features X_train: 53, X_test: 53
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                       class_weight='balanced_subsample', criterion='entropy',
                       max_depth=None, max_features='log2', max_leaf_nodes=None,
                    

Amount of features X_train: 53, X_test: 53
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=89, verbose=0,
                       warm_start=False)
Amount of features X_train: 53, X_test: 53


Iteration 127 of 144
Cohen's kappa score         0.40605155391143943
Accuracy score              0.9543802071791894
Recall score                0.7226231078907184
Precision score             0.3016792026372697

------------------------------------------------------

params: {'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_spli

Amount of features X_train: 53, X_test: 53


Iteration 132 of 144
Cohen's kappa score         0.4062406743295781
Accuracy score              0.9541092619336894
Recall score                0.7276124785985122
Precision score             0.3010523446879343

------------------------------------------------------

params: {'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_split': 2, 'class_weight': None, 'n_estimators': 100}
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=89, verbose=0,
                       warm_start=False)


Precision score             0.301531031532495

------------------------------------------------------

params: {'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_split': 10, 'class_weight': 'balanced', 'n_estimators': 300}
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=-1, oob_score=False, random_state=89, verbose=0,
                       warm_start=False)
Amount of features X_train: 53, X_test: 53
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=None, max_features

params: {'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_split': 10, 'class_weight': None, 'n_estimators': 200}
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=-1, oob_score=False, random_state=89, verbose=0,
                       warm_start=False)
Amount of features X_train: 53, X_test: 53
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, 

In [13]:
sorted(params_and_score,key=lambda x:(-x[1],x[0]))

[({'criterion': 'gini',
   'max_depth': None,
   'max_features': 'sqrt',
   'min_samples_split': 10,
   'class_weight': None,
   'n_estimators': 300},
  0.4297846249129419),
 ({'criterion': 'gini',
   'max_depth': 35,
   'max_features': 'sqrt',
   'min_samples_split': 10,
   'class_weight': None,
   'n_estimators': 300},
  0.42912782104665137),
 ({'criterion': 'gini',
   'max_depth': None,
   'max_features': 'sqrt',
   'min_samples_split': 10,
   'class_weight': 'balanced_subsample',
   'n_estimators': 300},
  0.4285003160562595),
 ({'criterion': 'gini',
   'max_depth': 35,
   'max_features': 'sqrt',
   'min_samples_split': 10,
   'class_weight': 'balanced_subsample',
   'n_estimators': 300},
  0.42844265073097276),
 ({'criterion': 'gini',
   'max_depth': None,
   'max_features': 'sqrt',
   'min_samples_split': 2,
   'class_weight': None,
   'n_estimators': 200},
  0.428351471627776),
 ({'criterion': 'gini',
   'max_depth': None,
   'max_features': 'sqrt',
   'min_samples_split': 10,
 